In [1]:
import argparse
import operator
import py_trees
import sys

import py_trees.console as console

class Nested(object):
    """
    A more complex object to interact with on the blackboard.
    """
    def __init__(self):
        self.foo = "bar"

    def __str__(self):
        return str({"foo": self.foo})


class BlackboardWriter(py_trees.behaviour.Behaviour):
    """
    Custom writer that submits a more complicated variable to the blackboard.
    """
    def __init__(self, name="Writer"):
        super().__init__(name=name)
        self.blackboard = self.attach_blackboard_client()
        self.blackboard.register_key(key="dude", access=py_trees.common.Access.READ)
        self.blackboard.register_key(key="spaghetti", access=py_trees.common.Access.WRITE)

        self.logger.debug("%s.__init__()" % (self.__class__.__name__))

    def update(self):
        """
        Write a dictionary to the blackboard and return :data:`~py_trees.common.Status.SUCCESS`.
        """
        self.logger.debug("%s.update()" % (self.__class__.__name__))
        try:
            unused = self.blackboard.dude
        except KeyError:
            pass
        try:
            unused = self.blackboard.dudette
        except AttributeError:
            pass
        try:
            self.blackboard.dudette = "Jane"
        except AttributeError:
            pass
        self.blackboard.spaghetti = {"type": "Carbonara", "quantity": 1}
        self.blackboard.spaghetti = {"type": "Gnocchi", "quantity": 2}
        try:
            self.blackboard.set("spaghetti", {"type": "Bolognese", "quantity": 3}, overwrite=False)
        except AttributeError:
            pass
        return py_trees.common.Status.SUCCESS


class ParamsAndState(py_trees.behaviour.Behaviour):
    """
    A more esotoric use of multiple blackboards in a behaviour to represent
    storage of parameters and state.
    """
    def __init__(self, name="ParamsAndState"):
        super().__init__(name=name)
        # namespaces can include the separator or may leave it out
        # they can also be nested, e.g. /agent/state, /agent/parameters
        self.parameters = self.attach_blackboard_client("Params", "parameters")
        self.state = self.attach_blackboard_client("State", "state")
        self.parameters.register_key(
            key="default_speed",
            access=py_trees.common.Access.READ
        )
        self.state.register_key(
            key="current_speed",
            access=py_trees.common.Access.WRITE
        )

    def initialise(self):
        try:
            self.state.current_speed = self.parameters.default_speed
        except KeyError as e:
            raise RuntimeError("parameter 'default_speed' not found [{}]".format(str(e)))

    def update(self):
        if self.state.current_speed > 40.0:
            return py_trees.common.Status.SUCCESS
        else:
            self.state.current_speed += 1.0
            return py_trees.common.Status.RUNNING


def create_root():
    root = py_trees.composites.Sequence("Blackboard Demo")
    set_blackboard_variable = py_trees.behaviours.SetBlackboardVariable(
        name="Set Nested", variable_name="nested", variable_value=Nested()
    )
    write_blackboard_variable = BlackboardWriter(name="Writer")
    check_blackboard_variable = py_trees.behaviours.CheckBlackboardVariableValue(
        name="Check Nested Foo",
        check=py_trees.common.ComparisonExpression(
            variable="nested.foo",
            value="bar",
            operator=operator.eq
        )
    )
    params_and_state = ParamsAndState()
    root.add_children([
        set_blackboard_variable,
        write_blackboard_variable,
        check_blackboard_variable,
        params_and_state
    ])
    return root


In [2]:
py_trees.logging.level = py_trees.logging.Level.DEBUG

In [3]:
py_trees.blackboard.Blackboard.enable_activity_stream(maximum_size=100)

In [4]:
blackboard = py_trees.blackboard.Client(name="Configuration")

In [5]:
blackboard.register_key(key="dude", access=py_trees.common.Access.WRITE)

In [6]:
blackboard.register_key(key="/parameters/default_speed", access=py_trees.common.Access.WRITE)

In [7]:
blackboard.dude = "Bob"

In [8]:
blackboard.parameters.default_speed = 30.0

In [9]:
root = create_root()

[DEBUG] Writer               : BlackboardWriter.__init__()


In [11]:
root.setup_with_descendants()

In [13]:
# if args.render:
#    py_trees.display.render_dot_tree(root, with_blackboard_variables=False)
#    sys.exit()
#if args.render_with_blackboard_variables:
#    py_trees.display.render_dot_tree(root, with_blackboard_variables=True)
#    sys.exit()

In [14]:
root.setup_with_descendants()

In [15]:
unset_blackboard = py_trees.blackboard.Client(name="Unsetter")

In [16]:
unset_blackboard.register_key(key="foo", access=py_trees.common.Access.WRITE)

In [17]:
print("\n--------- Tick 0 ---------\n")


--------- Tick 0 ---------



In [18]:
root.tick_once()

[DEBUG] Blackboard Demo      : Sequence.tick()
[DEBUG] Set Nested           : SetBlackboardVariable.tick()
[DEBUG] Set Nested           : SetBlackboardVariable.stop(Status.INVALID->Status.SUCCESS)
[DEBUG] Writer               : BlackboardWriter.tick()
[DEBUG] Writer               : BlackboardWriter.update()
[DEBUG] Writer               : BlackboardWriter.stop(Status.INVALID->Status.SUCCESS)
[DEBUG] Check Nested Foo     : CheckBlackboardVariableValue.tick()
[DEBUG] Check Nested Foo     : CheckBlackboardVariableValue.update()
[DEBUG] Check Nested Foo     : CheckBlackboardVariableValue.stop(Status.INVALID->Status.SUCCESS)
[DEBUG] ParamsAndState       : ParamsAndState.tick()


In [19]:
print(py_trees.display.unicode_tree(root, show_status=True))

{-} Blackboard Demo [*]
    --> Set Nested [✓]
    --> Writer [✓]
    --> Check Nested Foo [✓] -- 'nested.foo' comparison succeeded [v: bar][e: bar]
    --> ParamsAndState [*]



In [20]:
print(py_trees.display.unicode_blackboard())

Blackboard Data
    /dude                    : Bob
    /foo                     : -
    /nested                  : {'foo': 'bar'}
    /parameters/default_speed: 30.0
    /spaghetti               : {'type': 'Gnocchi', 'quantity': 2}
    /state/current_speed     : 31.0



In [21]:
print(py_trees.display.unicode_blackboard(display_only_key_metadata=True))

Blackboard Clients
    /dude                     : Configur... (w), Writer (r)
    /foo                      : Unsetter (w)
    /nested                   : Set Nested (w), Check Ne... (r)
    /parameters/default_speed : Configur... (w), Params (r)
    /spaghetti                : Writer (w)
    /state/current_speed      : State (w)



In [22]:
unset_blackboard.unset("foo")

False

In [23]:
print(py_trees.display.unicode_blackboard_activity_stream())

Blackboard Activity Stream
    /dude                     : INITIALISED   | Configuration    | → Bob
    /parameters/default_speed : INITIALISED   | Configuration    | → 30.0
    /nested                   : INITIALISED   | Set Nested       | → {'foo': 'bar'}
    /dude                     : READ          | Writer           | ← Bob
    /dudette                  : ACCESS_DENIED | Writer           | ✕ client has no read/write access
    /dudette                  : ACCESS_DENIED | Writer           | ✕ client has no read/write access
    /spaghetti                : INITIALISED   | Writer           | → {'type': 'Carb...
    /spaghetti                : WRITE         | Writer           | → {'type': 'Gnoc...
    /spaghetti                : NO_OVERWRITE  | Writer           | ⦸ {'type': 'Gnoc...
    /nested                   : READ          | Check Nested Foo | ← {'foo': 'bar'}
    /parameters/default_speed : READ          | Params           | ← 30.0
    /state/current_speed      : INITIALISED   | 